In [1]:
import numpy as np
import pandas as pd
from scipy import stats

## Early Data Analysis

In [2]:
df = pd.read_csv("AB_test_data.csv")
df["purchase_TF"] = df["purchase_TF"] * 1

In [3]:
# Here we split the dataframe into two dataframes for two variants

df_A = df[df["Variant"] == "A"]
df_B = df[df["Variant"] == "B"]

In [4]:
# Check the date for both variants

print(df_A["date"].min(), df_A["date"].max())
print(df_B["date"].min(), df_B["date"].max())

2019-08-01 2020-08-30
2020-08-01 2020-08-30


We have 13 months of Variant A data and 1 month of Variant B data.

In [5]:
# Check if each id is unique

len(df["id"].unique())

130000

If there are ids that are both in variant A and B groups, the assumption for two sample population would not be met.<br>
<br>
If there are duplicates, the 13-month Variant A data and 1-month Variant B data will not be independent and therefore we won't be able to use 13-month Variant A data but only 1-month Variant A data in two-sample t test. <br>
If there aren't duplicates, the 13-month Variant A data and 1-month Variant B data will be independent and therefore we will be able to use 13-month Variant A data in two-sample t test. <br>
<br>
It turns out that each id here is unique, meaning that we can use all data in two-sample t test.

## A/B test

In [6]:
n1 = len(df_B)
n2 = len(df_A)
print("The sample sizes for two variant groups are {},{}. Both are large enough.".format(n1, n2))

The sample sizes for two variant groups are 5000,125000. Both are large enough.


In [7]:
p1hat = sum(df_B["purchase_TF"])/len(df_B)
p2hat = sum(df_A["purchase_TF"])/len(df_A)

### Two sample t-test with all data

Since assumptions of two-sample t test are met, we can conduct a two-sample t test using all data.

Denote by p1, p2 the population conversion rate of Variant B and Variant A.<br>
Our hypothesis test is: <br>
<br>
H0: p1 - p2 = 0 <br>
Ha: p1 - p2 > 0 ,with a significant rate of 0.05.

To calculate test statistic, we need to know the following parameters: <br>
**p1hat**: Sample conversion rate of Variant B <br>
**p2hat**: Sample conversion rate of Variant A <br>
**pchat**: Combined estimate of population conversion rate <br>
**n1**: Sample size of Variant B group <br>
**n2**: Sample size of Variant A group

In [8]:
# Calculate pchat

pchat = (n1*p1hat + n2*p2hat)/(n1+n2)
print("The combined estimate of population conversion rate is {}.".format(pchat))

The combined estimate of population conversion rate is 0.15065384615384617.


In [9]:
# Calculate test statistic

z_score = (p1hat - p2hat)/(pchat*(1-pchat)*(1/n1 + 1/n2))
print("The test statistic is {}.".format(z_score))

The test statistic is 1013.8601308862007.


In [10]:
# Calculate the p-value

p_value = stats.norm.sf(abs(z_score))
print("The p-value is {}.".format(p_value))

The p-value is 0.0.


Since our p-value < 0.05, we reject our null hypothesis and conclude that Alternative B improved the conversion rate.

### Two sample t-test with 1-month data

In [11]:
# Get Variant A data in Auguest 2020

date_caliper = "2020-08-01"
df_Acut = df_A[df_A['date'] >= date_caliper]
print(df_Acut["date"].min(), df_Acut["date"].max())

2020-08-01 2020-08-30


For comparison, we conduct a two-sample t test using only data in August 2020.

Denote by p1, p2 the population conversion rate of Variant B and Variant A.<br>
Our hypothesis test is: <br>
<br>
H0: p1 - p2 = 0 <br>
Ha: p1 - p2 > 0 ,with a significant rate of 0.05.

To calculate test statistic, we need to know the following parameters: <br>
**p1hat**: Sample conversion rate of Variant B <br>
**p2hat**: Sample conversion rate of Variant A <br>
**pchat**: Combined estimate of population conversion rate <br>
**n1**: Sample size of Variant B group <br>
**n2**: Sample size of Variant A group <br>
<br>
In this test, **p1hat**, **n1** will be the same as the last test.<br>
We shall focus on how different **p2hat** and **n2** affect the values of **pchat** and our **test statistic**.

In [12]:
n1_2 = len(df_B)
n2_2 = len(df_Acut)
print("The sample sizes for two variant groups are {},{}.".format(n1_2, n2_2))

The sample sizes for two variant groups are 5000,5000.


In [13]:
p1hat_2 = sum(df_B["purchase_TF"])/len(df_B)
p2hat_2 = sum(df_Acut["purchase_TF"])/len(df_Acut)
print("The sample conversion rate of 13-month data is {} while that of 1-month data is {}.".format(p2hat, p2hat_2))

The sample conversion rate of 13-month data is 0.149616 while that of 1-month data is 0.1548.


We can see that **p2hat** we got with 1 month data is greater than the **p2hat** of 13 month data. <br>
This might be due to the fact that 1-month data only captures information of Auguest while 13-month data captures the information of the whole year.

In [14]:
# Calculate pchat

pchat_2 = (n1_2*p1hat_2 + n2_2*p2hat_2)/(n1_2+n2_2)
print("The combined estimate of population conversion rate is {}.".format(pchat_2))
print("Our estimate last time with 13-month data is {}.".format(pchat))

The combined estimate of population conversion rate is 0.1657.
Our estimate last time with 13-month data is 0.15065384615384617.


We can see that **pchat** we got this time is larger than last time.<br>
<br>
By breaking down the algorithm of pchat, we could easily find that the greatest effect came from a different **n2**.<br>
The value of **n1** and **n2** determines the weight of **p1hat** and **p2hat** in calculating pchat. <br>
In our last test with 13-month data, the weight of p1hat is **0.04**. <br>
In our current test with 1-month data, the weight of p1hat is **0.5**. <br>
<br>
Therefore, we got a larget pchat this time, which will impact our test statistic and p-value.

In [15]:
# Calculate test statistic

z_score_2 = (p1hat_2 - p2hat_2)/(pchat_2*(1-pchat_2)*(1/n1_2 + 1/n2_2))
print("The test statistic is {}.".format(z_score_2))
print("Our test statistic last time with 13-month data is {}.".format(z_score))

The test statistic is 394.2318883541082.
Our test statistic last time with 13-month data is 1013.8601308862007.


In [16]:
# Calculate the p-value

p_value_2 = stats.norm.sf(abs(z_score_2))
print("The p-value is {}.".format(p_value_2))

The p-value is 0.0.


Although p-value this time is still approximately 0 and rejects the null hypothesis. We need pay attention to the change in **z score**.<br>
<br>
In hypothesis tests, at a significant level of 0.05, we reject H0 if z score > 1.645. <br>
In our last test with 13-month data, the z score is **1013.86**. <br>
In our current test with 1-month data, the z score is **394.23**. <br>
<br>
We can see that, in this booking.com case, we are **more likely to reject H0** when using the 13-month data.<br>
If one uses the 13-month data for testing, he/she will be more likely to conclude that Variant B improved conversion rates.<br>
If one uses the 1-month data for testing, he/she will be more likely to conclude that Variant B didn't improve conversion rates.<br>
<br>
In this case, our conclusion didn't change. But this does indicate that **using different samples might lead to different test conclusions**.

### One sample t-test with all data

Aside from using different samples to represent Variant A population in two-sample t tests, we also like to compare two-sample t test to one-sample t test.<br>
<br>
For one-sample t test, we assume that the population conversion rate of Variant B is the same as that of Variant A, i.e., our champion alternative.<br>
Also, we notice that we don't need to check if two samples are independent since only one sample is used in one-sample t test.

In [17]:
p = sum(df_A["purchase_TF"])/len(df_A)
print("Our population conversion rate is {}, which is also our conversion rate assumption for Alternative B.".format(p))

Our population conversion rate is 0.149616, which is also our conversion rate assumption for Alternative B.


In [18]:
n = len(df_B)
print("The sample size for Variant B groups is {}.".format(n))

The sample size for Variant B groups is 5000.


Denote by p the population conversion rate of Variant B.<br>
Our hypothesis test is: <br>
<br>
H0: p = 0.1496 <br>
Ha: p > 0.1496 ,with a significant rate of 0.05.

To calculate test statistic, we need to know the following parameters: <br>
**phat**: Sample conversion rate of Variant B, **equals to p1hat** in two-sample t test <br>
**p**: Assumed population conversion rate of Variance B, **equals to p2hat** in two-sample t test <br>
**n**: Sample size of Variant B group, **equals to n1** in two-sample t test <br>
Less parameters are needed for one-sample t test than two-sample t test.

In [19]:
phat = sum(df_B["purchase_TF"])/len(df_B)
print("The sample conversion rate of Variant B is {}.".format(phat))

The sample conversion rate of Variant B is 0.1766.


In [20]:
# Calculate test statistic

z_score_1 = (phat - p)/(p*(1-p)/n)
print("The test statistic is {}.".format(z_score_1))
print("The test statistic of the two-sample t test is {}.".format(z_score))

The test statistic is 1060.4329470067144.
The test statistic of the two-sample t test is 1013.8601308862007.


The test statistic of the one-sample t test is greater than that of the two-sample t test.<br>
<br>
The z score function in two-sample t test is (p1hat - p2hat)/(pchat$*$(1-pchat)(1/n1 + 1/n2)).<br>
The z score function in one-sample t test with equivalent parameters is (p1hat - p2hat)/(p2hat$*$(1-p2hat)/n1).<br>
We can see that two z score functions are very similar.<br>


**Given that the test statistic of one-sample and two-sample tests are also very close and one-sample t test needs less parameters, we wonder if, under certain situations, two-sample t test is equivalent to one-sample t test.**<br>
<br>
There is no difference in the numerator part.<br>
In the demonimator part, pchat = 0.04$*$p1hat + 0.96$*$p2hat, and the weights of p1hat and p2hat are determined by sample sizes n1, n2.<br>
Also, as n2 approaches infinity, (1/n1 + 1/n2) approaches 1/n1.<br>
<br>
From comparisons above, we can see that the sample sizes n1 and n2 are the influencers of the two-sample t test.<br>

And this leads to our conclusions:<br>
<br>
**When the control group sample size n2 is much greater than the treatment group sample size n1, i.e., n2 >> n1, two-sample t test could be considered as equivalent to one-sample t test.**<br>
<br>
**Vice versa, when n2 is not much greater than n1, one-sample t test should not be used as a replacement of two-sample t test.**

In [21]:
# Calculate the p-value

p_value_1 = stats.norm.sf(abs(z_score_1))
print("The p-value is {}.".format(p_value_1))

The p-value is 0.0.


In conclusion to A/B test, all our hypothesis tests come to the conclusion that **Alternative B improved conversion rates over Alternative A**.<br>
Moreover, given our comparisons between hypothesis tests, **one-sample t test is optimal test** for the *booking.com* case.

## Solving for Optimal Sample Size

Since we use one-sided one-sample t test, we use a different optimizal sample size estimation algorithm.

n* = $(t\alpha + t\beta)^2$$p(1-p)\over\delta^2$

In [22]:
t_alpha = stats.norm.ppf(0.95)
t_beta = stats.norm.ppf(0.8)
p = sum(df_A["purchase_TF"])/len(df_A)
delta = 0.05

Using all Variant A data as our prior knowledge, we have the estimate conversion rate p = 0.1496.<br>
Also, let's assume we want our test to accurately detect the improvement when the true improvement $\delta$ = 0.05.

In [23]:
# Calculate the optimal sample size

nstar = round((t_alpha + t_beta)**2*p*(1-p)/delta**2)
print("The optimal sample size for a one-sided one-sample test with significant rate of 0.95 and a power of 0.2 is {}.".format(nstar))

The optimal sample size for a one-sided one-sample test with significant rate of 0.95 and a power of 0.2 is 315.


In [30]:
# Build a function that automatically conduct hypothesis test with one random sample.

def sample_test(n, tstar, estimate):
    
    '''
    The input is the sample size, t statistic at significant level, and estimate for the null hypothesis.
    The output is the test statistic and whether the null hypothesis is rejected.
    '''

    df_sample = df_B.sample(n = n)
    p = estimate
    phat = sum(df_sample["purchase_TF"])/nstar
    z = (phat-p)/(p*(1-p)/nstar)
    
    if abs(z) >= tstar:
        rejection = True
    else:
        rejection = False
    
    return z, rejection

In [36]:
# Conduct the test 10 times and report the result

test_result = [sample_test(nstar,t_alpha,p) for i in range(10)]
df_10test = pd.DataFrame(test_result, columns = ["Z score", "Reject H0"])
df_10test

,Z score,Reject H0
0,93.302380,True
1,54.003798,True
2,93.302380,True
3,14.705215,True
4,187.618978,True
5,14.705215,True
6,101.162096,True
7,54.003798,True
8,54.003798,True
9,6.845499,True


In [26]:
# Try the test 100 times and report the result

test_result = [sample_test(n,t_alpha,p) for i in range(100)]
df_100test = pd.DataFrame(test_result, columns = ["Z score", "Reject H0"])
df_100test["Reject H0"].value_counts()

True     97
False     3
Name: Reject H0, dtype: int64

Among the 10 tests in our first trial, all 10 tests rejected H0.<br>
Among the 100 tests in our second trial, 97 tests rejected H0 and 3 tests failed to reject H0.

## Sequential Probability Ratio Test

In [49]:
# Modify the function so that it returns the sample

def sample_test2(n, tstar, estimate):
    
    '''
    The input is the sample size, t statistic at significant level, and estimate for the null hypothesis.
    The output is the test statistic and whether the null hypothesis is rejected.
    '''

    df_sample = df_B.sample(n = n)
    p = estimate
    phat = sum(df_sample["purchase_TF"])/nstar
    z = (phat-p)/(p*(1-p)/nstar)
    
    if abs(z) >= tstar:
        rejection = True
    else:
        rejection = False
    
    return z, rejection, sample

In [52]:
# Rerun 10 samples and store the samples

z_list = []
rejection_list = []
df_list = []

for i in range(10):
    z, rejection, sample = sample_test2(nstar,t_alpha,p)
    z_list.append(z)
    rejection_list.append(rejection)
    df_list.append(sample)

In [54]:
# Show the test results of the 10 samples

df_test = pd.DataFrame()
df_test["Z score"] = z_list
df_test["Rejection"] = rejection_list
df_test

,Z score,Rejection
0,171.899545,True
1,22.564932,True
2,14.705215,True
3,164.039828,True
4,-1.014218,False
5,77.582947,True
6,85.442663,True
7,38.284365,True
8,61.863514,True
9,-8.873934,True


Our hypothesis test for SQRT is: <br>
<br>
H0: p = 0.1496 <br>
Ha: p = 0.18 ,with a significant rate of 0.05 and power of 0.8.

In [127]:
p0 = p
p1 = 0.18
print("The estimated conversion rate of our null hypothesis is {} and that of our alternative hypothesis is {}.".format(p0, p1))

The estimated conversion rate of our null hypothesis is 0.149616 and that of our alternative hypothesis is 0.18.


In [128]:
# Calculate log-likelihood changes for each new observation

log_pos = np.log(p1/p0)
log_neg = np.log((1-p1)/(1-p0))
print("Each observation of conversion will change the log-likelihood by {}.\nEach observation of non conversion will change the log-likelihood by {}.".format(log_pos, log_neg))

Each observation of conversion will change the log-likelihood by 0.18488483919711945.
Each observation of non conversion will change the log-likelihood by -0.03638367191699427.


In [106]:
# Calculate Wald boundaries

alpha = 0.05
beta = 0.2
a_upper = np.log((1-beta)/alpha)
b_lower = np.log(beta/(1-alpha))
print("We fail to reject H0 when log-likelihood is greater than {}.\nWe reject H0 when log-likelihood is smaller than {}.".format(a_upper, b_lower))

We fail to reject H0 when log-likelihood is greater than 2.772588722239781.
We reject H0 when log-likelihood is smaller than -1.5581446180465497.


In [113]:
# Build a function that performs SQRT on our sample

def sqrt_test(df, upper, lower):
    
    '''
    The input of the function is one sample dataframe, the upper bound, and the lower bound for the test.
    The output of the function is the number of observation that stops the test, test result, and log-likelihood value.
    '''
    
    conversion = df.reset_index()["purchase_TF"]
    log_lik = 0
    num = len(conversion)
    rejection = None
    
    for i in range(len(conversion)):
        if conversion[i] == 1:
            log_lik += log_pos
        elif conversion[i] == 0:
            log_lik += log_neg
        if log_lik >= upper:
            num = i+1
            rejection = True
            break
        elif log_lik <= lower:
            num = i+1
            rejection = False
            break
        
    return num, rejection, log_lik

In [155]:
df_sqrt10 = pd.DataFrame([sqrt_test(df) for df in df_list], columns = ["iter", "reject H0","log-likelihood"])
df_sqrt10

,iter,reject H0,log-likelihood
0,240,True,2.773881
1,315,None,-0.397431
2,315,None,-0.618700
3,218,True,2.910517
4,128,False,-1.559351
5,315,None,1.151448
6,315,None,1.372717
7,315,None,0.045106
8,315,None,0.708911
9,208,False,-1.593554


In [159]:
avg_iter10 = df_sqrt10["iter"].mean()
print("The average iteration number, including those that didn't come to conclusions, is {}.".format(avg_iter10))

The average iteration number, including those that didn't come to conclusions, is 268.4.


Among the 10 samples we had, only 4 of them stopped the test prior to using the full sample.<br>
It seems that we needed more samples for sequential probability ratio test than one-sample t test.<br>
Unlike one-sample t tests, our available SQRTs are not consistent in results: 2 rejected H0 and 2 failed to reject H0.<br>
<br>
I think that the result of SQRT is largely affected by my choice of alternative proportion.<br>
Proportion closer to the null should need larger sample than proportion far from the null.